Todos los Imports

In [0]:
import PIL
from PIL import Image, ImageOps
import os
import fnmatch
import shutil
import sys
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from tensorflow import keras
import h5py
from enum import Enum


Crear Directorios

In [0]:
##CrearDirectorios
os.mkdir("./img")
os.mkdir("./img_resize")
os.mkdir("./entrenamiento")
os.mkdir("./entrenamiento/bulbasaur")
os.mkdir("./entrenamiento/pikachu")
os.mkdir("./entrenamiento/charmander")
os.mkdir("./entrenamiento/squirtle")
os.mkdir("./validacion")
os.mkdir("./validacion/bulbasaur")
os.mkdir("./validacion/pikachu")
os.mkdir("./validacion/charmander")
os.mkdir("./validacion/squirtle")

Crear DATASET a partir de IMG (PNG) # Solo mueve a Entrenamiento

In [0]:
pathImagenesOriginales = './img/'
pathImagenesReducidas =  './img_resize/'
listaImagenes = fnmatch.filter(os.listdir(pathImagenesOriginales), '*')
no_png = ".png"
bul = "bul"
pik = "pik"
ch= "ch"
sq = "sq"

for i in listaImagenes:
    if  no_png not in i:
        listaImagenes.remove(i)
    
for x in listaImagenes: 

    img = Image.open(pathImagenesOriginales+x)
    new_img = img.resize((28,28))
    new_img = ImageOps.grayscale(new_img)
    new_img.save(pathImagenesReducidas + x)

    if bul in x:
      shutil.move(pathImagenesReducidas + x, './entrenamiento/bulbasaur')
    if pik in x:
      shutil.move(pathImagenesReducidas + x, './entrenamiento/pikachu')
    if ch in x:
      shutil.move(pathImagenesReducidas + x, './entrenamiento/charmander')
    if sq in x:
      shutil.move(pathImagenesReducidas + x, './entrenamiento/squirtle')

##os.rmdir('./')
os.listdir("./entrenamiento")

Creacion y entrenamiento de la RED

In [0]:
tf.compat.v1.disable_eager_execution() #Esto si lo corren LOCAL quiza lo tengan que comentar
K.clear_session()

data_entrenamiento = './entrenamiento/'
data_validacion = './entrenamiento/' # Cambiar

"""
Parameters
"""
epocas=20
longitud, altura = 28, 28
batch_size = 32
pasos = 1000
validation_steps = 10
clases = 4
lr = 0.0004


##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

cnn = Sequential()
cnn.add(Flatten(input_shape=(28, 28,3)))
cnn.add(Dense(512, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])




cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

Print Grafico + Print ID del Array

In [0]:
plt.plot(cnn.history.history['accuracy'], label='accuracy')
plt.plot(cnn.history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1.3])
plt.legend(loc='lower right')
print(entrenamiento_generador.class_indices)
print("\n")

Predict Automatico


In [0]:
longitud, altura = 28, 28
cnn = keras.models.load_model('./modelo/modelo.h5')
cnn.load_weights('./modelo/pesos.h5')
class TipoAnimal(Enum):
    BULBASAUR = 'BULBASAUR'
    PIKACHU = 'PIKACHU'
    CHARMANDER = 'CHARMANDER'
    SQUIRTLE = 'SQUIRTLE'
    DESCONOCIDO = 'DESCONOCIDO'


def predict(file: str) -> TipoAnimal:

    x = load_img(file, target_size=(longitud, altura))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)

    array = cnn.predict(x)
    result = array[0]
    answer = np.argmax(result)

    if answer == 4:
        return TipoAnimal.DESCONOCIDO
    elif answer == 0:
        return TipoAnimal.BULBASAUR
    elif answer == 1:
        return TipoAnimal.CHARMANDER
    elif answer == 2:
        return TipoAnimal.PIKACHU
    elif answer == 3:
        return TipoAnimal.SQUIRTLE

    return TipoAnimal.DESCONOCIDO


# PRUEBAS
aciertos_bul = 0
veces_bul = 22
for vez_bul in range(veces_bul):

    imagen_bulbasaur = f'./entrenamiento/bulbasaur/bul{vez_bul}.png'
    if predict(imagen_bulbasaur) == TipoAnimal.BULBASAUR:
        aciertos_bul += 1

aciertos_char = 0
veces_char = 22
for vez_char in range(veces_char):

    imagen_charmander = f'./entrenamiento/charmander/ch{vez_char}.png'
    if predict(imagen_charmander) == TipoAnimal.CHARMANDER:
        aciertos_char += 1

aciertos_pik = 0
veces_pik = 22

for vez_pik in range(veces_pik):

    imagen_pikachu = f'./entrenamiento/pikachu/pik{vez_pik}.png'
    if predict(imagen_pikachu) == TipoAnimal.PIKACHU:
        aciertos_pik += 1

aciertos_sq = 0
veces_sq = 22

for vez_sq in range(veces_sq):

    imagen_squirtle = f'./entrenamiento/squirtle/sq{vez_sq}.png'
    if predict(imagen_squirtle) == TipoAnimal.SQUIRTLE:
        aciertos_sq += 1

print('\n\nRESULTADOS:\n')
print(
    f'BULBAUSAR -> aciertos:{aciertos_bul}, pruebas:{veces_bul}, porcentaje:{100 * aciertos_bul / veces_bul}%')
print(
    f'CHARMANDER -> aciertos:{aciertos_char}, pruebas:{veces_char}, porcentaje: {100 * aciertos_char / veces_char}%')
print(
    f'PIKACHU -> aciertos:{aciertos_pik}, pruebas:{veces_pik}, porcentaje: {100 * aciertos_pik / veces_pik}%')
print(
    f'SQUIRTLE -> aciertos:{aciertos_sq}, pruebas:{veces_sq}, porcentaje: {100 * aciertos_sq / veces_sq}%')

Predict Manual

In [0]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import h5py

longitud, altura = 28, 28
cnn = keras.models.load_model('./modelo/modelo.h5')
cnn.load_weights('./modelo/pesos.h5')

def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Bulbasaur")
  elif answer == 1:
    print("pred: Charmander")
  elif answer == 2:
    print("pred: Pikachu")
  elif answer == 3:
    print("pred: Squirtle")

  return answer

predict("./entrenamiento/pikachu/pik16.png")

Funcion para resize y greyscale manual

In [0]:

img = Image.open("./pik2.png")
new_img = img.resize((28,28))
new_img = ImageOps.grayscale(new_img)
new_img.save("pik2B.png")